In [1]:
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
data = pd.read_csv('US_nn_plus.csv')
profit_mean = data['profit'].mean()
profit_std = data['profit'].std()
#Delete column profit
data = data.drop(columns=['profit'])
class CustomDataset(Dataset):
    def __init__(self, data):
        self.input = torch.tensor(data[data.columns[5:]].values, dtype=torch.float32)
        self.output = torch.tensor(data[data.columns[0:5]].values, dtype=torch.float32)
    def __getitem__(self, index):
        x = self.input[index]
        y_recom = self.output[index, :]
        return x, y_recom
    def __len__(self):
        return len(self.input)
dataset = CustomDataset(data)
train, val = torch.utils.data.random_split(dataset, [0.8,0.2])
train_loader = DataLoader(train, batch_size=64, shuffle=True)
val_loader = DataLoader(val, batch_size=len(val), shuffle=True)


In [2]:
class res_block(nn.Module):
  def __init__(self, in_features, out_features,dropout=0.5):
      super(res_block, self).__init__()
      self.fc1 = nn.Linear(in_features, out_features)
      self.bn1 = nn.BatchNorm1d(out_features)
      self.dropout1 = nn.Dropout(dropout)
      self.fc2 = nn.Linear(out_features, out_features)
      self.bn2 = nn.BatchNorm1d(out_features)
      self.dropout2 = nn.Dropout(dropout)
      self.fc3 = nn.Linear(in_features, out_features)
  def forward(self, x):
      residual = x.clone()
      x = F.relu(self.dropout1(self.bn1(self.fc1(x))))
      x = self.dropout2(self.bn2(self.fc2(x)))
      x = x + self.fc3(residual)
      return F.relu(x)

class simple_model(nn.Module):
  def __init__(self,drop=0.3):
    super(simple_model, self).__init__()
    self.bn_start = nn.BatchNorm1d(13)
    self.block1 = res_block(13, 32, drop)
    self.block2 = res_block(32,64,drop)
    self.block3 = res_block(64,128,drop)
    self.block4 = res_block(128,64,drop)
    self.block5 = res_block(64,32,drop)
    self.block6 = res_block(32,5,drop)
  def forward(self, x):
    x = self.bn_start(x)
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    x = self.block4(x)
    x = self.block5(x)
    x = self.block6(x)
    return x

class small_model(nn.Module):
  def __init__(self,drop=0.3):
    super(small_model, self).__init__()
    self.bn_start = nn.BatchNorm1d(13)
    self.block1 = res_block(13, 32, drop)
    self.block2 = res_block(32,32,drop)
    self.block3 = res_block(32,32,drop)
    self.block4 = res_block(32,32,drop)
    self.fn = nn.Linear(32,5)
  def forward(self, x):
    x = self.bn_start(x)
    x = self.block1(x)
    x = self.block2(x)
    x = self.block3(x)
    x = self.block4(x)
    x = self.fn(x)
    return x

def loss_fn(x, y_recom):
    recom = F.softmax(x, dim=1)
    return F.binary_cross_entropy(recom, y_recom)

model = small_model(0.2).to('cuda')

print(sum(p.numel() for p in model.parameters()))

12159


In [3]:
writer = SummaryWriter()
exmaple_batch = next(iter(train_loader))
x, y = exmaple_batch
writer.add_graph(model, x.to('cuda'))

In [28]:
del model
torch.cuda.empty_cache()

In [4]:
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=0.001)
#Use simple scheduler
sceduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
def train_one_epoch(model, optimizer, train_loader):
    model.train()
    for x, y_recom in train_loader:
        x = x.to('cuda')
        y_recom = y_recom.to('cuda')
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output,  y_recom)
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    return loss.item()/len(train_loader)
def val_loss(model, val):
    model.eval()
    with torch.no_grad():
        for x, y_recom in val:
            x = x.to('cuda')
            y_recom = y_recom.to('cuda')
            output = model(x)
            loss = loss_fn(output, y_recom)
        return loss.item()

In [10]:
model.to('cuda')
for i in range(300,500):
    print('Epoch:', i)
    loss_train =  train_one_epoch(model, optimizer, train_loader)
    loss_val =  val_loss(model, val_loader)
    writer.add_scalar('Loss/train', loss_train, i)
    writer.add_scalar('Loss/val', loss_val, i)
    print('Train loss:', loss_train)
    print('Val loss:', loss_val)
    

Epoch: 300
Train loss: 0.036402422934770584
Val loss: 0.42413660883903503
Epoch: 301
Train loss: 0.03671134263277054
Val loss: 0.4154580235481262
Epoch: 302
Train loss: 0.04266427084803581
Val loss: 0.419908344745636
Epoch: 303
Train loss: 0.03509564325213432
Val loss: 0.41586148738861084
Epoch: 304
Train loss: 0.03931846842169762
Val loss: 0.4156135022640228
Epoch: 305
Train loss: 0.03549457713961601
Val loss: 0.43117016553878784
Epoch: 306
Train loss: 0.029728049412369728
Val loss: 0.44474926590919495
Epoch: 307
Train loss: 0.04399974271655083
Val loss: 0.42615726590156555
Epoch: 308
Train loss: 0.026687443256378174
Val loss: 0.4371587336063385
Epoch: 309
Train loss: 0.017793603241443634
Val loss: 0.4417608380317688
Epoch: 310
Train loss: 0.0360056534409523
Val loss: 0.44482433795928955
Epoch: 311
Train loss: 0.03838464617729187
Val loss: 0.43446874618530273
Epoch: 312
Train loss: 0.030972803011536598
Val loss: 0.4360280930995941
Epoch: 313
Train loss: 0.029922345653176308
Val loss: 

In [13]:
torch.save({"model":model.to('cpu'), "parameters":model.to('cpu').state_dict()}, 'model500.pth')

In [ ]:
# model.load_state_dict(torch.load('model.pth')['parameters'])

In [11]:
@torch.no_grad()
def recom_accuracy(x,y_recom):
  recom = F.softmax(model(x), dim=1)
  recom = torch.argmax(recom, dim=1)
  y_recom = torch.argmax(y_recom, dim=1)
  return torch.sum(recom == y_recom).item() / len(y_recom)
#Caclulate the accuracy of all datasets
model.eval()
model.to('cuda')
x_all = torch.tensor(data[data.columns[5:]].values, dtype=torch.float32).to('cuda')
y_recommend_all = torch.tensor(data[data.columns[:5]].values, dtype=torch.float32).to('cuda')
acc_sum = 0
for i in range(len(x_all)//32 + 1):
    torch.cuda.empty_cache()
    x = x_all[i*32:(i+1)*32]
    y_recommend = y_recommend_all[i*32:(i+1)*32]
    acc_sum += recom_accuracy(x, y_recommend)*x.shape[0]
print('Recommendation accuracy:', acc_sum/len(x_all))

#Calulate Val accuracy
torch.cuda.empty_cache()
for x_val, y_recommend_val in val_loader:
    x_val = x_val.to('cuda')
    y_recommend_val = y_recommend_val.to('cuda')
acc_sum_val = 0
for i in range(len(x_val)//32 + 1):
    torch.cuda.empty_cache()
    x = x_val[i*32:(i+1)*32]
    y_recommend = y_recommend_val[i*32:(i+1)*32]
    acc_sum_val += recom_accuracy(x, y_recommend)*x.shape[0]
print('Val Recommendation accuracy:', acc_sum_val/len(x_val))

#Calculate the top 2 accuracy
@torch.no_grad()
def top2_accuracy(x, y_recom):
    recom = F.softmax(model(x), dim=1)
    _, top2_indices = torch.topk(recom, k=2, dim=1)
    y_recom = torch.argmax(y_recom, dim=1)

    correct = 0
    for i in range(len(y_recom)):
        if y_recom[i] in top2_indices[i]:
            correct += 1

    return correct / len(y_recom)
acc_sum = 0
for i in range(len(x_all)//32 + 1):
    torch.cuda.empty_cache()
    x = x_all[i*32:(i+1)*32]
    y_recommend = y_recommend_all[i*32:(i+1)*32]
    acc_sum += top2_accuracy(x, y_recommend)*x.shape[0] 
print('Top 2 accuracy:', acc_sum/len(x_all))

#Calculate the top 2 accuracy of val
torch.cuda.empty_cache()
for x_val, y_recommend_val in val_loader:
    x_val = x_val.to('cuda')
    y_recommend_val = y_recommend_val.to('cuda')
acc_sum_val = 0
for i in range(len(x_val)//32 + 1):
    torch.cuda.empty_cache()
    x = x_val[i*32:(i+1)*32]
    y_recommend = y_recommend_val[i*32:(i+1)*32]
    acc_sum_val += top2_accuracy(x, y_recommend)*x.shape[0]
print('Val Top 2 accuracy:', acc_sum_val/len(x_val))

Recommendation accuracy: 0.8783333333333333
Val Recommendation accuracy: 0.55
Top 2 accuracy: 0.9566666666666667
Val Top 2 accuracy: 0.8


In [12]:
writer.close()